In [ ]:
!git clone https://github.com/wadi-sudo/track2023.git

In [ ]:
%cd track2023
!unzip data3.zip

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from tqdm import tqdm
import torchvision.models as models


def train(model, train_loader, optimizer, criterion, device):
    model.train()

    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        _, predicted = outputs.max(1)
        train_total += targets.size(0)
        train_correct += predicted.eq(targets).sum().item()
        train_loss += loss.item() * inputs.size(0)

    train_loss /= train_total
    train_acc = train_correct / train_total

    return train_loss, train_acc


def evaluate(model, val_loader, criterion, device):
    model.eval()

    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            _, predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()
            val_loss += loss.item() * inputs.size(0)

    val_loss /= val_total
    val_acc = val_correct / val_total

    return val_loss, val_acc

# Define the data transforms
transform = transforms.Compose([
    transforms.Resize((256, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
train_dataset = ImageFolder('/content/track2023/data-3classes/train', transform=transform)
val_dataset = ImageFolder('/content/track2023/data-3classes/test', transform=transform)

# Set up the data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)



# Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)




# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

# Define the device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define the number of training epochs
num_epochs = 10

# Train the model
for epoch in range(num_epochs):
    # Train for one epoch
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)

    # Evaluate on the validation set
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    # Print the results for this epoch
    print(f'Epoch {epoch+1}: Train loss: {train_loss:.4f}, Train accuracy: {train_acc:.4f}, '
          f'Val loss: {val_loss:.4f}, Val accuracy: {val_acc:.4f}')

# Save the model
model_path = '/content/model111.pt'
torch.save(model.state_dict(), model_path)
